In [1]:
%reset -f
from django_for_jupyter import init_django
init_django()

/home/javier/hd1/Codigo/JAVIER/test1/comparaprecios-1


[nltk_data] Downloading package stopwords to /home/javier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/javier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from precios.models import (Marcas, Articulos, Vendedores, Unifica, Settings, SiteURLResults)
from precios.pi_get import (
    reemplaza_palabras,
    replace_comma_in_degrees,
    get_palabras_con_numychar,
    extract_and_remove_weight_range_updated,
    remueveYGuarda,
    remueveYGuardaSinSplit,
    obtener_grados,
    obtener_talla,
    obtener_marca,
    get_unidadMedida,
    separate_numbers_from_text,
    to_lowercase,
    remove_some_chars,
    remove_stopwords,
    get_unidades2,
    get_unidades,
    get_dics,
    normalize,
)   

In [3]:
# Importando las librerías que vamos a utilizar
import html
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import textacy
from textacy.datasets import Wikipedia
from collections import Counter, defaultdict
import warnings; warnings.simplefilter('ignore')

In [4]:
import re, string, unicodedata
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

import  textacy
# import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

[nltk_data] Downloading package stopwords to /home/javier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/javier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:


# from textacy import preprocessing
# from textacy import extract
# nlp = textacy.load_spacy_lang("en_core_web_sm")

# ex = """
# 1. Normalización de Datos

# Primero, puedes intentar normalizar los nombres de los productos para que diferentes variantes del 
# mismo producto puedan identificarse como uno solo. Para esto, podrías usar técnicas de procesamiento 
# de lenguaje natural (NLP), como la tokenización, el análisis morfológico y la eliminación de palabras vacías (stop words).
# """
 
# # # Remove Punctuation
# # rm_punc = preprocessing.remove.punctuation(ex)
 
# # Remove Whitespace
# rm_wsp = preprocessing.normalize.whitespace(ex)
 
# # # Remove Punctuation and Whitespace both
# # rm_all = preprocessing.normalize.whitespace(rm_punc)



In [6]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words



def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas




In [7]:
PALABRAS_INUTILES, \
SUJIFOS_NOMBRE, \
ean_13_site_ids, \
UMEDIDAS, \
UNIDADES, \
PACKS, \
TALLAS, \
COLORES, \
ENVASES, \
marcas, \
listamarcas,\
marcasistema,\
sin_marca, \
listamarcas, \
campoMarcaObj = get_dics()

lista_articulos = SiteURLResults.objects.all()
# lista_articulos = lista_articulos.filter(nombre__istartswith='pisco')
lista_articulos = lista_articulos.filter(nombre__istartswith='whiskey jack daniel')

lista_articulos = lista_articulos[0:50]
for articulo in lista_articulos:
    anombre     = articulo.nombre
    # marca       = articulo.marca
    newmarca_str = articulo.marca
    unidades    = articulo.unidades
    medida_um   = articulo.medida_um
    medida_cant = articulo.medida_cant
    print('----------------------')

        
    print(f'0->Nombre={anombre} Marca={newmarca_str} Unidades={unidades} medida_um={medida_um} medida_cant={medida_cant} ')

    anombre                 = html.unescape(anombre)
    anombre                 = reemplaza_palabras(anombre)
    print('1->', anombre)

    anombre, newmarca_str   = obtener_marca(anombre, newmarca_str)

    anombre                 = separate_numbers_from_text(anombre)
    # print('2->', anombre)

    anombre = replace_comma_in_degrees(anombre)

    words = nltk.word_tokenize(anombre)
    print('3->', words)
    
    words = remove_non_ascii(words)
    print('4->', words)

    words = normalize(words)
    # print('4->', words)

    nombre = ' '.join(words)
    
    inutiles, nombre        = remueveYGuardaSinSplit(PALABRAS_INUTILES, nombre, remover=True, todos=True)
    
    agregar_sufijos, nombre = remueveYGuarda(SUJIFOS_NOMBRE, nombre, " ", remover=True, todos=True)
    
    talla, nombre           = obtener_talla(nombre, TALLAS)

    envase, nombre          = remueveYGuarda(ENVASES, nombre, " ", remover=True, todos=True)

    nombre                  = replace_comma_in_degrees(nombre)
    nombre,  grados         = obtener_grados(nombre)
    
    dimension, nombre       = extract_and_remove_weight_range_updated(nombre)
    nombre, medida_cant, medida_um, dimension = get_unidadMedida(nombre, UMEDIDAS)

    words = nltk.word_tokenize(nombre)
    nombre = ' '.join(words)
    

    
    print(f'10->Nombre={nombre} Marca={newmarca_str} Unidades={unidades} grados={grados} medida_um={medida_um} medida_cant={medida_cant}')


----------------------
0->Nombre=whiskey jack daniel&#39;s honey 35° 750 cc Marca=jack daniel's Unidades=1.0 medida_um= medida_cant=0.0 
1-> whiskey jack daniel's honey 35° 750 cc
3-> ['whiskey', 'honey', '35°', '750', 'cc']
4-> ['whiskey', 'honey', '35', '750', 'cc']
10->Nombre=whiskey honey 35 Marca=jack daniel's Unidades=1.0 grados=None medida_um=cc medida_cant=750.0
----------------------
0->Nombre=whiskey jack daniels 40° botella 1 l Marca=jack daniel's Unidades=1.0 medida_um= medida_cant=0.0 
1-> whiskey jack daniels 40° botella 1 l
3-> ['whiskey', 'jack', 'daniels', '40°', 'botella', '1', 'l']
4-> ['whiskey', 'jack', 'daniels', '40', 'botella', '1', 'l']
10->Nombre=whiskey jack daniels 40 Marca=jack daniel's Unidades=1.0 grados=None medida_um=cc medida_cant=1000.0
----------------------
0->Nombre=whiskey jack daniels 40° botella 1 l Marca=jack daniel's Unidades=1.0 medida_um= medida_cant=0.0 
1-> whiskey jack daniels 40° botella 1 l
3-> ['whiskey', 'jack', 'daniels', '40°', 'bot